#Training bert model

In [10]:
import csv

# Define data to write into CSV
data = [
    ["word1"],  # Header row
    ["apple"],
    ["banana"],
    ["grape"], ["strawberry"], ["blueberry"], ["blackberry"], ["raspberry"], ["apricot"], ["lemon"], ["lime"], ["cantaloupe"]
]

# Specify file path
filename = "words1.csv"

# Writing to CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"CSV file '{filename}' created successfully.")


CSV file 'words1.csv' created successfully.


In [11]:
import csv

# Define data to write into CSV
data = [
    ["word2"],  # Header row
    ["fruit"],
    ["yellow"],
    ["berry"],["fruit"], ["yellow"], ["berry"], ["citrus"], ["tropical"],["berry"], ["stone fruit"], ["stone fruit"]
]

# Specify file path
filename = "words2.csv"

# Writing to CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print(f"CSV file '{filename}' created successfully.")


CSV file 'words2.csv' created successfully.


combine the dataset

In [12]:
import torch
from transformers import BertTokenizer, BertModel from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

class SemanticSimilarity:
    def __init__(self, model_name='bert-base-uncased'):
        # Load pre-trained BERT model and tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def get_embedding(self, word):
        # Tokenize and get the embeddings for a single word
        inputs = self.tokenizer(word, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Take the mean of the token embeddings (ignoring padding)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings

    def calculate_similarity(self, word1, word2):
        # Get embeddings for both words
        embedding1 = self.get_embedding(word1)
        embedding2 = self.get_embedding(word2)
        
        # Calculate cosine similarity
        similarity = cosine_similarity(embedding1.numpy(), embedding2.numpy())
        return similarity[0][0]

    def compare_csv(self, file1, file2, column_name1, column_name2):
        # Read CSV files
        df1 = pd.read_csv(file1)
        df2 = pd.read_csv(file2)

        # Check if the specified columns exist
        if column_name1 not in df1.columns or column_name2 not in df2.columns:
            raise ValueError(f"Columns '{column_name1}' or '{column_name2}' not found in the CSV files.")

        # Initialize a list to store similarity results
        similarities = []

        # Loop over words from both columns and calculate similarity
        for word1 in df1[column_name1]:
            for word2 in df2[column_name2]:
                sim = self.calculate_similarity(word1, word2)
                similarities.append((word1, word2, sim))
        
        # Convert results to a DataFrame
        similarity_df = pd.DataFrame(similarities, columns=[column_name1, column_name2, 'Similarity'])
        return similarity_df

# Example usage:
semantic_similarity = SemanticSimilarity()

# Assuming 'words1.csv' and 'words2.csv' have columns 'word1' and 'word2' respectively
similarity_results = semantic_similarity.compare_csv('words1.csv', 'words2.csv', 'word1', 'word2')

# Print or save the result
print(similarity_results)


          word1        word2  Similarity
0         apple        fruit    0.809608
1         apple       yellow    0.810922
2         apple        berry    0.843468
3         apple        fruit    0.809608
4         apple       yellow    0.810922
..          ...          ...         ...
116  cantaloupe       citrus    0.415452
117  cantaloupe     tropical    0.372999
118  cantaloupe        berry    0.343872
119  cantaloupe  stone fruit    0.509545
120  cantaloupe  stone fruit    0.509545

[121 rows x 3 columns]


In [13]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

class SemanticSimilarity:
    def __init__(self, model_name='bert-base-uncased'):
        # Load pre-trained BERT model and tokenizer
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)

    def get_embedding(self, word):
        # Tokenize and get the embeddings for a single word
        inputs = self.tokenizer(word, return_tensors='pt', truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Take the mean of the token embeddings (ignoring padding)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings

    def calculate_similarity(self, word1, word2):
        # Get embeddings for both words
        embedding1 = self.get_embedding(word1)
        embedding2 = self.get_embedding(word2)
        
        # Calculate cosine similarity
        similarity = cosine_similarity(embedding1.numpy(), embedding2.numpy())
        return similarity[0][0]

    def compare_csv(self, file1, file2, column_name1, column_name2, output_file):
        # Read CSV files
        df1 = pd.read_csv(file1)
        df2 = pd.read_csv(file2)

        # Check if the specified columns exist
        if column_name1 not in df1.columns or column_name2 not in df2.columns:
            raise ValueError(f"Columns '{column_name1}' or '{column_name2}' not found in the CSV files.")

        # Initialize a dictionary to store the top 5 similar words for each word
        results = []

        # Loop over words from the first file
        for word1 in df1[column_name1]:
            similarities = []
            for word2 in df2[column_name2]:
                sim = self.calculate_similarity(word1, word2)
                similarities.append((word2, sim))
            
            # Get the top 5 similar words
            top_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)[:5]
            for similar_word, sim_score in top_similarities:
                results.append((word1, similar_word, sim_score))
        
        # Create a DataFrame for the results
        results_df = pd.DataFrame(results, columns=['Input Word', 'Similar Word', 'Similarity'])
        
        # Write results to a new CSV file
        results_df.to_csv(output_file, index=False)
        print(f"Results written to {output_file}")

# Example usage:
semantic_similarity = SemanticSimilarity()

# Assuming 'words1.csv' and 'words2.csv' have columns 'word1' and 'word2' respectively
output_file = "top_similarities.csv"
semantic_similarity.compare_csv('words1.csv', 'words2.csv', 'word1', 'word2', output_file)


Results written to top_similarities.csv


In [3]:
import pandas as pd
df = pd.read_csv('top_similarities.csv')
print(df.head(10))


  Input Word Similar Word  Similarity
0      apple     tropical    0.847134
1      apple        berry    0.843468
2      apple        berry    0.843468
3      apple        berry    0.843468
4      apple       citrus    0.823191
5     banana     tropical    0.899704
6     banana        berry    0.894563
7     banana        berry    0.894563
8     banana        berry    0.894563
9     banana       citrus    0.869619
